In [1]:
%load_ext autoreload
%autoreload 2

In [116]:
import jax
# jax.config.update("jax_enable_x64", True)

import jax.numpy as jnp
import jax.random as jr
from jax.tree_util import Partial

import hugegp as gp
import hugegp_cuda as hgc
import jaxkd as jk

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import time
from tqdm import tqdm

rng = jr.key(137)

In [117]:
rng, key = jr.split(rng)
points = jr.normal(key, shape=(10_000, 2))
offsets = gp.make_offsets(len(points), 1000, 1.3, 5000)
points, neighbors, indices = gp.build_jax_graph(points, offsets=offsets, k=10, cuda=True)
gp.check_graph(points, neighbors, offsets, indices)

In [118]:
rng, key = jr.split(rng)
xi = jr.normal(key, shape=(len(points),))
values = gp.generate(points, neighbors, offsets, gp.matern_cov, xi)

xi

Array([-0.6495131 ,  1.4007365 , -1.4015812 , ..., -1.2978343 ,
       -0.65852773, -0.2735266 ], dtype=float32)

In [119]:
xi2 = gp.generate_inv(points, neighbors, offsets, gp.matern_cov, values)
xi2

Array([-0.6495131 ,  1.4007365 , -1.4015813 , ..., -1.2978345 ,
       -0.65852845, -0.27352697], dtype=float32)

In [123]:
gp.generate_logdet(points, neighbors, offsets, gp.matern_cov)

Array(-22549.541, dtype=float32)

In [126]:
gp.generate_dense_logdet(points, gp.matern_cov)

Array(-23259.057, dtype=float32)